In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import us
import copy
import tqdm.auto as tqdm
import geopandas as gpd
import numpy as np
import pandas as pd
from shapefiles import shapefiles, school_district_shapefiles, ce_to_name
from relationship import full_relationships, create_relationships
from stats_for_shapefile import compute_statistics_for_shapefile, compute_summed_shapefile_all_keys, sum_keys
from create_website import (
    full_shapefile,
    shapefile_without_ordinals,
    add_ordinals,
    get_statistic_categories,
    create_page_jsons,
)
from election_data import with_election_results
from urbanstats.features.extract_data import feature_data
import matplotlib.pyplot as plt
import matplotlib as mpl

from urbanstats.data.gpw import *
from urbanstats.data.population_urban_area import *

from collections import defaultdict, Counter
import requests

from shapefiles import CONGRESSIONAL_DISTRICTS, COUNTIES, county_cross_cd, usda_county_type, shapefiles

/home/kavi/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
import io


In [4]:
from urbanstats.osm.buses import national_stops
from urbanstats.osm.trains import *
from urbanstats.data.circle import *

In [5]:
ghs = load_full_ghs()

In [6]:
# """

# ok here's something terrible (not v vectorized). Technically quadratic


# the ellipse centered at (x0, y0) has equation (x - x0)^2 cos^2 y0 + (y - y0)^2 <= r^2

# - precompute the function cumulative_map[y, x] = sum_{i <= x} map[y, i]
# - binary search over r:
#     - iterate over y
#         - let pop[x] = 0
#         - iterate over y' from y - r to y + r
#             - compute rx' as rx' = sqrt (r^2 - (y' - y)^2) * sec y
#             - vectorized calculate: pop[x] += cumulative_map[y', x + rx'] - cumulative_map[y', x - rx']
#         - see if any elements of pop are > P
# """

In [7]:
# np.where(ghs > 3e5)

In [8]:
# chunk = 20
# ghs_small = ghs.reshape(ghs.shape[0] // chunk, chunk, ghs.shape[1] // chunk, chunk).sum((1, 3))

In [9]:
# circles_no = non_overlapping_circles(ghs_small, 10**8, limit=1000)

In [10]:
# circles, circle_map = overlapping_circles(ghs_small, 10**8, limit=1000)

In [11]:
# plt.figure(dpi=100, figsize=(20, 10))
# plot_circles(ghs_small, circles_no, reduce=1)

In [12]:
# plt.figure(dpi=100, figsize=(40, 20))
# plot_overlapping_circles(ghs_small, circles, circle_map, reduce=1) 

In [13]:
circles_no = non_overlapping_circles(ghs, 10**7, limit=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

value 2 low 1 high None


0it [00:00, ?it/s]

found None
value 4 low 2 high None


  0%|          | 0/121 [00:00<?, ?it/s]

...

found (4841, 22665)
value 80.0 low 64 high 96.0


  0%|          | 0/10008 [00:00<?, ?it/s]

found None
value 88.0 low 80.0 high 96.0


  0%|          | 0/10308 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
circles, circle_map = overlapping_circles(ghs, 10**7, limit=1000)

In [ ]:
plt.figure(dpi=200, figsize=(20, 10))
plot_circles(ghs, circles_no, reduce=10, lw=1)
plt.xlim(20, 40)
plt.ylim(30, 40)

In [ ]:
_, (y, x) = circles_no[-1]
lat_from_row_idx(y), lon_from_col_idx(x)